<a href="https://colab.research.google.com/github/ChrisOmics/MachineLearning/blob/master/CNN_DNN_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Here I create a DNN followed by a DNN for image classification from a large dataset 
## 1 - Intro
We will be using the  CIFAR-10 dataset
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 
We will train a DNN followed by a CNN and test the accuracy.
## 2 - Downloading the data and installing dependencies

In [ ]:

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.1-{platform}-linux_x86_64.whl torchvision

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [ ]:

!pip install --upgrade --force-reinstall -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
!pip3 install https://download.pytorch.org/whl/cu80/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
import torch
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)


    100% |████████████████████████████████| 484.0MB 65.7MB/s 
    100% |████████████████████████████████| 71kB 6.3MB/s 
    100% |████████████████████████████████| 17.3MB 1.8MB/s 
    100% |████████████████████████████████| 2.0MB 13.1MB/s 
jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.15 which is incompatible.
google-colab 1.0.0 has requirement six~=1.11.0, but you'll have six 1.12.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.46 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
    100% |████████████████████████████████| 530.8MB 30kB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0
Torch 1.0.1.post2 CUDA 8.0.61
Device: cuda:0
c

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import torch.utils.data as td
import random,time, sys

## 3 - Downlaoding Training and Test Data

In [ ]:

def cifar_loaders(batch_size, shuffle_test=False): 
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.225, 0.225, 0.225])
    train = datasets.CIFAR10('./', train=True, download=True, 
        transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]))
    test = datasets.CIFAR10('./', train=False, 
        transform=transforms.Compose([transforms.ToTensor(), normalize]))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size,
        shuffle=shuffle_test, pin_memory=True)
    return train_loader, test_loader
  

batch_size = 64
test_batch_size = 64

train_loader, _ = cifar_loaders(batch_size)
_, test_loader = cifar_loaders(test_batch_size)



0it [00:00, ?it/s]

100%|█████████▉| 170483712/170498071 [07:44<00:00, 108057.84it/s]

Files already downloaded and verified


## 4 - Creating DNN and testing accuracy

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__() #load super class for training data
        self.fc1 = nn.Linear(3072, 1000) #defining fully connected with input 3072 (3x32x32 pixels) and output 320
        self.fc2 = nn.Linear(1000, 540) #defining fully connected with input 1000 and output 540
        self.fc3 = nn.Linear(540, 300) #defining fully connected
        self.fc4 = nn.Linear(300, 270)
        self.fc5 = nn.Linear(270, 150)
        self.fc6 = nn.Linear(150, 50)
        self.fc7 = nn.Linear(50, 10)
        self.relu = nn.ReLU() #defining Rectified Linear Unit as activation function

    def forward(self, x): #feed forward
        layer1 = x.view(-1, 3072) #make it flat in one dimension from 0 - 784
        layer2 = self.relu(self.fc1(layer1)) #layer2 = layer1 -> fc1 -> relu
        layer3 = self.relu(self.fc2(layer2)) #layer3 = layer2 -> fc2 -> relu
        layer4 = self.relu(self.fc3(layer3)) #layer4 = layer3 -> fc2 -> relu
        layer5 = self.relu(self.fc4(layer4)) #layer2 = layer1 -> fc1 -> relu
        layer6 = self.relu(self.fc5(layer5)) #layer3 = layer2 -> fc2 -> relu
        layer7 = self.relu(self.fc6(layer6))
        layer8 = self.relu(self.fc7(layer7))
        return layer8 


In [ ]:
net=DNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epoch_loss_all=[]
net.to(device)
for epoch in range(10):  # loop over the dataset multiple times

    epoch_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        epoch_loss += float(loss.item())
    print('[epoch %d, batch %5d] loss: %.8f' %(epoch + 1, i + 1, epoch_loss/(i + 1)))
    epoch_loss_all.append(epoch_loss/(i + 1))
print('Finished Training')



[epoch 1, batch   782] loss: 2.30274210
[epoch 2, batch   782] loss: 2.30170476
[epoch 3, batch   782] loss: 2.29998546
[epoch 4, batch   782] loss: 2.29248874
[epoch 5, batch   782] loss: 2.24742164
[epoch 6, batch   782] loss: 2.20125571
[epoch 7, batch   782] loss: 2.18846756
[epoch 8, batch   782] loss: 2.17097623
[epoch 9, batch   782] loss: 2.14683260
[epoch 10, batch   782] loss: 2.12488738
Finished Training


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 25 %


## 5 - Creating CNN and testing accuracy

In [ ]:
class CNN(nn.Module):
	def __init__(self):
		super(CNN, self).__init__() #load super class for training data
		self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=2, padding=1) #Convolutional modul: input 3, output 10, kernel 5
		self.conv2 = nn.Conv2d(10, 12, kernel_size=2, stride=1, padding=1) #Convolutional modul: input 10, output 20, kernel 5
		self.conv3 = nn.Conv2d(12, 8, kernel_size=2, stride=2, padding=2)
		self.conv4 = nn.Conv2d(8, 64, kernel_size=3, stride=1, padding=0) 
		self.relu = nn.ReLU() #activation relu modul
		self.fc1 = nn.Linear(4096, 200) #Fully Connected modul: input, output
		self.fc2 = nn.Linear(200, 50)# Fully Connected modul: input, output
		self.fc3 = nn.Linear(50, 10)# Fully Connected modul: input, output

	def forward(self, x): #feed forward
		layer1 = self.relu((self.conv1(x))) # layer1 = x -> conv1 -> maxpool -> relu
		layer2 = self.relu((self.conv2(layer1))) # layer1 = x -> conv1 -> maxpool -> relu
		layer3 = self.relu((self.conv3(layer2)))# layer1 = x -> conv1 -> maxpool -> relu
		layer4 = self.relu((self.conv4(layer3))) # layer1 = x -> conv1 -> maxpool -> relu
		#print(layer4.size())
    
		layer4 = layer4.view(-1, 4096)
		layer5 = self.relu(self.fc1(layer4)) #layer4 = layer3 -> fc1 -> relu
		layer6 = self.relu(self.fc2(layer5)) #layer4 = layer3 -> fc1 -> relu
		layer7 = self.fc3(layer6) #layer5 = layer4 -> fc2
		return layer7
        #return F.log_softmax(layer5) #softmax activation to layer5


In [ ]:
def inout(inw=32, kernel=2, stride=2, pad=1):
  outwidth = (inw + 2 * pad - (kernel - 1) -1)/stride + 1
  print("outwidth =  %i" %outwidth)

  
inout(32, 3, 2, 1)
inout(16, 2, 1, 1)
inout(17, 2, 2, 2)
inout(10, 3, 1, 0)

outwidth =  16
outwidth =  17
outwidth =  10
outwidth =  8


In [ ]:
net2=CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net2.parameters(), lr=0.001, momentum=0.9, nesterov=True, weight_decay=0.0005)
epoch_loss_all=[]
net2.to(device)
for epoch in range(100):  # loop over the dataset multiple times
    
    epoch_loss = 0.0
    start = time.time()
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        epoch_loss += float(loss.item())
    print('[epoch %d] loss: %.8f time: %f' %(epoch + 1, epoch_loss/(i + 1), time.time() - start))
    epoch_loss_all.append(epoch_loss/(i + 1))
print('Finished Training')



[epoch 1] loss: 2.30390039 time: 19.161966
[epoch 2] loss: 2.30144435 time: 19.300143
[epoch 3] loss: 2.28721505 time: 19.110650
[epoch 4] loss: 2.12755660 time: 19.309396
[epoch 5] loss: 1.96700541 time: 19.202543
[epoch 6] loss: 1.85664379 time: 19.118961
